In [1]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install -q pymysql

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-libmysqlclient-dev is already the newest version (1.0.4).
default-libmysqlclient-dev set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
     |████████████████████████████████| 43 kB 1.1 MB/s 


In [2]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import pymysql

In [ ]:
host = '****.***.ap-northeast-2.rds.amazonaws.com'
port = 3306
username = '***'
password = '****'
db = '*****'

conn = pymysql.connect(host=host, port=port, user=username, password=password, db=db)

In [4]:
# 유튜브 댓글
sql_state = 'SELECT target_candi, content FROM youtube_comment A JOIN youtube_list B ON A.youtube_id=B.id'

youtube_df = pd.read_sql_query(sql_state, conn)

In [6]:
print(youtube_df['target_candi'].value_counts()) # 1: 이재명, 2: 윤석열
youtube_df.head()

2    55480
1    50668
Name: target_candi, dtype: int64


,target_candi,content
0,1,빨간불 이 수준높은 댓글러들 안보이나 무식한 양아치 티 내지 말고 나서지 말지
1,1,아휴 소름 형수한테 욕하는 모습
2,1,이재명후보 너무 똑똑합니다 격하게 지지합니다
3,1,이후보님 세상사를 보는 안목과 통찰력이 정말 대단하신 것 같음 자본시장 문제점과 대...
4,1,이방송보고 구독했어요


In [ ]:
# 뉴스 댓글
sql_state = 'SELECT comment FROM news_comment A JOIN news_list B ON B.title=A.title'

news_df = pd.read_sql_query(sql_state, conn)
conn.close()

In [ ]:
news_df.head()

In [7]:
!pip install -q mxnet
!pip install -q gluonnlp pandas tqdm
!pip install -q sentencepiece
!pip install -q transformers>=4.8.1
!pip install -q torch

     |████████████████████████████████| 47.3 MB 1.7 MB/s 
     |████████████████████████████████| 344 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 5.4 MB/s 


In [8]:
!pip install -q git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 4.5 MB 42.4 MB/s 
     |████████████████████████████████| 8.5 MB 19.9 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 127 kB 47.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [9]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/My Drive/Colab Notebooks/soyo_project/Election/'

Mounted at /content/drive


In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader

import gluonnlp as nlp

from tqdm import tqdm

In [11]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [12]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [14]:
# GPU 사용 시
device = torch.device("cuda:0")
print(device)

bertmodel, vocab = get_pytorch_kobert_model()

cuda:0
/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [15]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [45]:
# 학습 때와 달리 label 파라미터가 필요없으므로 삭제
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, bert_tokenizer, max_len,
                pad, pair):
    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

    self.sentences = [transform([i[sent_idx]]) for i in dataset]

  def __getitem__(self, i):
    return self.sentences[i]

  def __len__(self): # valid_length를 위해 필요한 내장 함수
    return len(self.sentences)

In [31]:
class BERTClassifier(nn.Module):
  def __init__(self,
                bert,
                hidden_size = 768,
                num_classes = 3, # softmax 사용 <- binary일 경우는 2
                dr_rate=None,
                params=None):
    
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate
              
    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)
  
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

model = BERTClassifier(bertmodel, dr_rate=0.5).to(device) # 모델 초기화

* model.pt 자체를 불러오면 에러 발생 -> 이는 새로운 데이터에 따라 모델이 조금이라도 달라지만 문제가 발생한다고 함
* https://discuss.huggingface.co/t/bertencoder-object-has-no-attribute-gradient-checkpointing/11207

In [32]:
# 저장된 모델 불러오기
model.load_state_dict(torch.load(PATH + 'model_state.pt')) 
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [53]:
# setting parameters
max_len = 128

mapping = {0: '중립', 1: '긍정', 2: '부정'}

In [46]:
unseen_test = youtube_df[['content']]
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, tok, max_len, True, False)

test_input = DataLoader(test_set, batch_size=1, num_workers=2) # colab은 max num works가 2

result_list = []

for batch_id, (token_ids, valid_length, segment_ids) in enumerate(tqdm(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)

  max_vals, max_indices = torch.max(out, 1)
  result_list.append(max_indices) # 중립: 0, 긍정: 1, 부정: 2

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  6%|▌         | 5924/106148 [02:33<42:53, 38.94it/s]Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 232, in _feed
    close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent

In [60]:
# mapping 변환
new_list = []
for ele in result_list:
  new_list.append(mapping[int(ele)])
new_list[:3]

['부정', '긍정', '긍정']

In [59]:
# 댓글 전처리 하지 않은 오리지널 버전 일단 저장하기로...
youtube_result = pd.concat([youtube_df, pd.DataFrame(new_list, columns=['sentiment'])], axis=1)
print(youtube_result.shape)
youtube_result.sample(10)

(106148, 3)


,target_candi,content,sentiment
59344,2,저도 그 생각이 딱 들었으나 실제 틱장애 친구들이 생각나서 그표현은 차마 안썼네요....,중립
51844,2,동문서답 듣다가 암걸리것다,부정
71999,2,MSCI는 너무 단순하게 보신듯 보다 많은 뜻이 있어 보입니다 .,긍정
90510,2,삼프로 수준이 이정도인줄<br>몰랐을듯요<br><br>틀튜브만 보신다는 소문이 있어...,부정
19653,1,나라 구한 애국채널이라 구독했습니다,중립
93122,2,"전 포기했어요,,,,,",중립
26127,1,경험도 있고 성과도 있고 소신도 있고 깡다구도 있으시고 인격적으로 완벽한 사람은 아...,중립
44302,1,비교자체가 안돼네?,부정
38438,1,안구건조증 인가 뭔가 눈이 지금 조금아프데요,중립
31521,1,a h ef http www youtu e com esults sea ch_que ...,긍정


In [61]:
youtube_result.to_csv(PATH + 'dataset/youtbue_sentiment.csv', index=False, encoding='utf-8-sig')